In [1]:
# !pip install -U insightface

import torch
# if torch.cuda.is_available():
#   print('gpu is available')
#   !pip install onnxruntime-gpu==1.9.0   # to use GPU
# else:
#   print('gpu is not available')
#   !pip install onnxruntime

In [ ]:
from insightface.app import FaceAnalysis


app = FaceAnalysis()
app.prepare(ctx_id=0, det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kali/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/kali/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0


In [ ]:
import os
import cv2
import numpy as np

dataset_path = '/home/kali/Desktop/face_cup/datasets/sample_lfw_images/'
files = os.listdir(dataset_path)

accepted_files = []
features = []
labels = []
reject_labels = []
counter = 1
for file in files:
  if not file.endswith('.jpg'):
    continue
  print('prosessing image %d: %s'%(counter,file))
    
  img = cv2.imread(os.path.join(dataset_path, file))
  faces = app.get(img)
  if len(faces) > 0:
    features.append(faces[0].normed_embedding)
    accepted_files.append(file)
    labels.append(int(file[1:3]))
  else:
    reject_labels.append(int(file[1:3]))
  
  counter+=1

labels += reject_labels # add images without face detection
print('number of images without face: %d' % len(reject_labels))

In [ ]:
from sklearn import metrics

def calculate_f_measure(labels_true, labels_pred, beta=1.0):
  [[tn, fp], [fn, tp]] = metrics.cluster.pair_confusion_matrix(labels_true, labels_pred)
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)
  f_measure = (beta*beta + 1)*precision*recall/(beta*beta*precision + recall)
  return f_measure

def evaluate_clustering(labels, predictions, beta=1.0):
  rand_index = metrics.rand_score(labels, predictions)
  nmi = metrics.normalized_mutual_info_score(labels, predictions)
  f_measure = calculate_f_measure(labels, predictions, beta=beta)
  print('Rand Index: %f, NMI: %f, F-measure: %f' % (rand_index, nmi, f_measure))
  return

In [5]:
from sklearn import cluster
k = 40 # set the number of clusters
y_pred = cluster.KMeans(n_clusters=k, random_state=0).fit_predict(features)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)

num of clusters: 63
Rand Index: 0.975764, NMI: 0.882530, F-measure: 0.708727


In [6]:


y_pred = cluster.DBSCAN(eps=1.14, min_samples=2).fit_predict(features)

# remove noise label from the output:
# (assign a separate cluster label for each noise image)
c_id = max(y_pred)+1
for i, y in enumerate(y_pred):
  if y == -1:
    y_pred[i] = c_id
    c_id += 1

y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)



num of clusters: 83
Rand Index: 0.994733, NMI: 0.959152, F-measure: 0.946191


In [7]:
y_pred = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=1.14, linkage='single').fit_predict(features)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)

num of clusters: 83
Rand Index: 0.994733, NMI: 0.959152, F-measure: 0.946191


In [9]:
import numpy as np

def match(template1, template2, metric='Euclidean'):
    # assert (template1.size == 512)
    # assert (template2.size == 512)
    assert (template1.size == template2.size)

    if metric == 'Euclidean':
        # Euclidean distance
        diff = np.subtract(template1, template2)
        return np.sqrt(np.sum(np.square(diff)))
    elif metric == 'Cosine':
        # Cosine Similarity
        return np.dot(template1, template2.T)
    else:
        raise Exception("input metric must be Euclidean or Cosine!")

In [10]:


# re-implement this part of code to reduce run-time:
n = len(features)
distance_matrix = np.empty([n, n])
for i, feature in enumerate(features):
  for j in range(i+1, n): # distance[i, j] == distance [j, i]
    distance_matrix[i, j] = match(feature, features[j])
    # distance_matrix[i, j] = match(feature, features[j], metric='Cosine') # precomputed --> a distance matrix is needed



In [11]:


# affinity='precomputed'
y_pred = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=1.14, linkage='single', affinity='precomputed').fit_predict(distance_matrix)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)



num of clusters: 83
Rand Index: 0.994733, NMI: 0.959152, F-measure: 0.946191
